In [28]:
#此為鐵達尼號存活率預測,老師作的
import pandas as pd
from urllib.request import urlretrieve
url = "https://github.com/Elwing-Chou/tibaml1027/raw/main/train.csv"
urlretrieve(url, "train.csv")
train = pd.read_csv("train.csv", encoding="utf-8")
url = "https://github.com/Elwing-Chou/tibaml1027/raw/main/test.csv"
urlretrieve(url, "test.csv")
test = pd.read_csv("test.csv", encoding="utf-8")

In [29]:
test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [34]:
# concat
datas = pd.concat([train, test], axis=0, ignore_index=True)#合併後一併處理,補植 ignore_index就是讓資料正常有幾個顯示出來
datas = datas.drop(["PassengerId", "Survived"], axis=1)#要想清楚位甚麼axis = 1跟= 0的差別 drop是丟掉甚麼的意思
datas

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...
1304,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
1305,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
1306,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
1307,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [35]:
# 篩選操作 Series/DataFrame[跟你的資料筆數依樣多的True/False]
s = datas.isna().sum()#判斷每個位子有沒有值,並計算出結果
s[s > 0].sort_values(ascending=False))#把為0的東西剃除

SyntaxError: ignored

In [36]:
#範例可以改 Flase 跟True
d1 = pd.DataFrame([
    [1, 2],
    [3, 4]
])
d1[[False, True]]

,0,1
1,3,4


In [37]:
most = datas["Embarked"].value_counts().idxmax()#idmax為最多出現的
datas["Embarked"] = datas["Embarked"].fillna(most)#只要有缺失的值就把她補成s
# 再看一下缺多少
s = datas.isna().sum()
s[s > 0].sort_values(ascending=False)

Cabin    1014
Age       263
Fare        1
dtype: int64

In [38]:
#完整的正規表示式
import re
def name(s):
    # .+任意一次以上 , \s*空白零次以上 (.+) \. .+
    # 比.好的 [a-zA-Z\s\(\)']
    pattern = r".+,\s*(.+)\..+"
    # s = "Braund, Mr. Owen Harris"
    pat = re.compile(pattern)
    return pat.match(s).group(1)
datas["Name"].apply(name)

0           Mr
1          Mrs
2         Miss
3          Mrs
4           Mr
         ...  
1304        Mr
1305      Dona
1306        Mr
1307        Mr
1308    Master
Name: Name, Length: 1309, dtype: object

In [39]:
def name(s):
    return s.split(",")[-1].split(".")[0].strip()#strip是用來把左右兩邊空白的都丟掉
datas["Name"] = datas["Name"].apply(name)

In [40]:
def cabin(c):
    if pd.isna(c):
        return c
    else:
        return c[0]
datas["Cabin"] = datas["Cabin"].apply(cabin)

In [41]:
dic = datas["Ticket"].value_counts()
datas["Ticket"] = datas["Ticket"].apply(lambda t:dic[t])

In [42]:
datas["Cabin"].value_counts

<bound method IndexOpsMixin.value_counts of 0       NaN
1         C
2       NaN
3         C
4       NaN
       ... 
1304    NaN
1305      C
1306    NaN
1307    NaN
1308    NaN
Name: Cabin, Length: 1309, dtype: object>

In [47]:
#datas.median()取中位數
med = datas.median().drop("Pclass")
datas = datas.fillna(med)
# 再看一下缺多少
s = datas.isna().sum()
s[s > 0].sort_values(ascending=False)

Cabin    1014
Name       34
dtype: int64

In [43]:
c = datas["Name"].value_counts()
c = c[c > 50]
def name2(n):
    if n in c:
        return n
    else:
        return None
datas["Name"] = datas["Name"].apply(name2)

In [50]:
datas = pd.get_dummies(datas)#dummies會自動幫你給欄位名稱,如果只有這行的話會有pclass
datas = pd.get_dummies(datas, columns=["Pclass"])#新增出一個Pcalss的欄位
datas

,Age,SibSp,Parch,Ticket,Fare,Name_Master,Name_Miss,Name_Mr,Name_Mrs,Sex_female,Sex_male,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3
0,22.0,1,0,1,7.2500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1
1,38.0,1,0,2,71.2833,0,0,0,1,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0
2,26.0,0,0,1,7.9250,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
3,35.0,1,0,2,53.1000,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0
4,35.0,0,0,1,8.0500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,28.0,0,0,1,8.0500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1
1305,39.0,0,0,3,108.9000,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0
1306,38.5,0,0,1,7.2500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1
1307,28.0,0,0,1,8.0500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1


In [ ]:
datas["Family"] = datas["SibSp"] + datas["Parch"]#提供一個Fimaly來提高它的機率
datas

In [53]:

# .iloc -> [1st row, 2nd row, 3rd....]
x = datas.iloc[:len(train)]
y = train["Survived"]
x_predict = datas.iloc[len(train):]

In [ ]:
x_predict


In [60]:
from sklearn.ensemble import  RandomForestClassifier
from sklearn.model_selection import GridSearchCV
params ={
    "n_estimators":range(21,200,2),
    "max_depth":range(5, 11)
}
clf = RandomForestClassifier()
search = GridSearchCV(clf, params,cv=10, n_jobs=-1)
search.fit(x,y)
print(search.best_score_)
print(search.best_params_)

0.842896379525593
{'max_depth': 7, 'n_estimators': 149}


In [71]:
import numpy as np
from sklearn.model_selection import cross_val_score
clf = RandomForestClassifier(n_estimators=149,max_depth=7)
scores = cross_val_score(clf, x, y, cv=10,n_jobs=-1)

print(scores)
print(np.average(scores))

[0.82222222 0.88764045 0.74157303 0.88764045 0.87640449 0.83146067
 0.85393258 0.78651685 0.88764045 0.83146067]
0.840649188514357


In [75]:
clf = RandomForestClassifier(n_estimators=149,max_depth=7)
clf.fit(x,y)
pre = clf.predict(x_predict)
result = pd.DataFrame({
    "PassengerId":test["PassengerId"],
    "Survived":pre
})
result.to_csv("rf.csv", encoding="utf-8", index=False)
result

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
